In [1]:
from ROOT import *
import uproot
import numpy as np
import pandas as pd
import itertools
import json

Welcome to JupyROOT 6.28/04


In [15]:
file_path = "/eos/user/z/zewang/HZGamma_data/run2UL/skimmed_ntuples"
bkg_names = ['ZG2JToG2L2J', 'WZ',  'TTGJets', 'DYJetsToLL', 'WW', 'TGJets', 'ZGToLLG', 'TT', 'EWKZ2J', 'ttWJets', 'ttZJets', 'WGToLNuG', 'ZZ']
sig_names = ['ggH', 'VBF', 'ZH', 'WminusH', 'WplusH', 'ttH']
tree = "zero_to_one_jet"
#tree = "two_jet"
#tree = "inclusive"
reweight_variables = ['Z_cos_theta', 'H_relpt', 'gamma_mvaID', 'gamma_relpt', 'gamma_ptRelErr']
norm_variables = ['H_mass', 'weight_central']
years = ['2016postVFP', '2016preVFP','2017','2018']
json_name = "/afs/cern.ch/work/z/zewang/private/HZGamma/HiggsZGammaAna/Plotter/Data/reweight_run2UL_{}.json".format(tree)

bins = {}
bins['Z_cos_theta'] = [[-1., -0.4] , [-0.4, 0.4], [0.4, 1.]]
bins['H_relpt'] = [[0., 0.05], [0.05, 0.1], [0.1, 1.]]
bins['gamma_mvaID'] = [[0.4, 0.8], [0.8, 0.9], [0.9, 1.]]
bins['gamma_relpt'] = [[0., 0.15], [0.15, 0.2], [0.2, 0.4]]
bins['gamma_ptRelErr'] = [[0.01, 0.02], [0.02, 0.03], [0.03, 0.08]]

In [16]:
data_list = []
for year in years:
    data = uproot.open("{}/data/{}.root".format(file_path, year))
    data_tree = data[tree]
    data_arrays = data_tree.arrays(reweight_variables+norm_variables, library="pd")
    data_list.append(data_arrays)
    #print(len(data_arrays))

data_run2 = pd.concat(data_list)

bkg_list = []
for year in years:
    for bkg in bkg_names:
        bkg = uproot.open("{}/{}/{}.root".format(file_path, bkg, year))
        bkg_tree = bkg[tree]
        bkg_arrays = bkg_tree.arrays(reweight_variables+norm_variables, library="pd")
        bkg_list.append(bkg_arrays)
        #print(len(data_arrays))
bkg_run2 = pd.concat(bkg_list)

In [17]:
#sum(bkg_run2["H_mass>120&H_mass<130"]["weight_central"])
bkg_SB = bkg_run2[((bkg_run2["H_mass"]>100) & ((bkg_run2["H_mass"]<120))) | ((bkg_run2["H_mass"]>130) & ((bkg_run2["H_mass"]<180)))]
data_SB = data_run2[((data_run2["H_mass"]>100) & ((data_run2["H_mass"]<120))) | ((data_run2["H_mass"]>130) & ((data_run2["H_mass"]<180)))]
norm_CR_MC = sum(bkg_SB["weight_central"])
norm_CR_data = sum(data_SB["weight_central"])
SF_norm = norm_CR_data/norm_CR_MC
print("Sideband NData: {}, Nbkg: {}, norm_factor = NData/Nbkg = {}".format(norm_CR_data, norm_CR_MC, SF_norm))

Sideband NData: 195409.0, Nbkg: 209281.8993820579, norm_factor = NData/Nbkg = 0.933711900441366


In [18]:

bin_all = []
reweights = {}
i=0
for ibin in itertools.product(bins['Z_cos_theta'], bins['H_relpt'], bins['gamma_mvaID'], bins['gamma_relpt'], bins['gamma_ptRelErr']):
    bin_all.append(ibin)
    reweights["bin_"+str(i)] = []
    Nbkg_SB_ibin = sum(bkg_SB[ (bkg_SB['Z_cos_theta'] > ibin[0][0]) & (bkg_SB['Z_cos_theta'] < ibin[0][1]) & (bkg_SB['H_relpt'] > ibin[1][0]) & (bkg_SB['H_relpt'] < ibin[1][1]) & (bkg_SB['gamma_mvaID'] > ibin[2][0]) & (bkg_SB['gamma_mvaID'] < ibin[2][1]) & (bkg_SB['gamma_relpt'] > ibin[3][0]) & (bkg_SB['gamma_relpt'] < ibin[3][1]) & (bkg_SB['gamma_ptRelErr'] > ibin[4][0]) & (bkg_SB['gamma_ptRelErr'] < ibin[4][1]) ]["weight_central"])
    Ndata_SB_ibin = sum(data_SB[ (data_SB['Z_cos_theta'] > ibin[0][0]) & (data_SB['Z_cos_theta'] < ibin[0][1]) & (data_SB['H_relpt'] > ibin[1][0]) & (data_SB['H_relpt'] < ibin[1][1]) & (data_SB['gamma_mvaID'] > ibin[2][0]) & (data_SB['gamma_mvaID'] < ibin[2][1]) & (data_SB['gamma_relpt'] > ibin[3][0]) & (data_SB['gamma_relpt'] < ibin[3][1]) & (data_SB['gamma_ptRelErr'] > ibin[4][0]) & (data_SB['gamma_ptRelErr'] < ibin[4][1]) ]["weight_central"])
    reweight_SFs_ibin = Ndata_SB_ibin/(Nbkg_SB_ibin*SF_norm)
    print("Ndata_SB_ibin:", Ndata_SB_ibin, "Nbkg_SB_ibin:", Nbkg_SB_ibin, "SFs:", reweight_SFs_ibin)
    reweights["bin_"+str(i)].append(ibin)
    reweights["bin_"+str(i)].append(reweight_SFs_ibin)

    i+=1

#data_SB[]

Ndata_SB_ibin: 163.0 Nbkg_SB_ibin: 163.37880803620092 SFs: 1.0685109745166366
Ndata_SB_ibin: 122.0 Nbkg_SB_ibin: 147.49228454694116 SFs: 0.8858855831133442
Ndata_SB_ibin: 337.0 Nbkg_SB_ibin: 448.4069650468349 SFs: 0.8049050589367428
Ndata_SB_ibin: 328.0 Nbkg_SB_ibin: 421.0357817210697 SFs: 0.8343378457773272
Ndata_SB_ibin: 286.0 Nbkg_SB_ibin: 394.59117658279985 SFs: 0.7762574285400358
Ndata_SB_ibin: 632.0 Nbkg_SB_ibin: 784.3633929130009 SFs: 0.8629524533086343
Ndata_SB_ibin: 354.0 Nbkg_SB_ibin: 421.519682163926 SFs: 0.8994406466122404
Ndata_SB_ibin: 263.0 Nbkg_SB_ibin: 325.7719241127863 SFs: 0.864627812728173
Ndata_SB_ibin: 282.0 Nbkg_SB_ibin: 371.3964495415197 SFs: 0.8132020533837578
Ndata_SB_ibin: 162.0 Nbkg_SB_ibin: 153.520017844538 SFs: 1.1301526514788824
Ndata_SB_ibin: 77.0 Nbkg_SB_ibin: 118.00599876646307 SFs: 0.6988335491494636


Ndata_SB_ibin: 134.0 Nbkg_SB_ibin: 172.423217930263 SFs: 0.832331168851527
Ndata_SB_ibin: 340.0 Nbkg_SB_ibin: 437.5023246250656 SFs: 0.8323110453066584
Ndata_SB_ibin: 207.0 Nbkg_SB_ibin: 340.0287053043314 SFs: 0.651991403447424
Ndata_SB_ibin: 231.0 Nbkg_SB_ibin: 298.71039683240315 SFs: 0.8282257847070915
Ndata_SB_ibin: 383.0 Nbkg_SB_ibin: 457.14181576946856 SFs: 0.8972943454493614
Ndata_SB_ibin: 209.0 Nbkg_SB_ibin: 288.0727156007858 SFs: 0.7770183325173133
Ndata_SB_ibin: 93.0 Nbkg_SB_ibin: 124.99659307939064 SFs: 0.7968413792225068
Ndata_SB_ibin: 301.0 Nbkg_SB_ibin: 468.66762899968813 SFs: 0.6878419259878727
Ndata_SB_ibin: 76.0 Nbkg_SB_ibin: 110.3509464729193 SFs: 0.737606331186583
Ndata_SB_ibin: 62.0 Nbkg_SB_ibin: 69.4135925811166 SFs: 0.9566085826883818
Ndata_SB_ibin: 858.0 Nbkg_SB_ibin: 1203.7001777671528 SFs: 0.7634068791788543
Ndata_SB_ibin: 217.0 Nbkg_SB_ibin: 282.09461000856544 SFs: 0.8238574088923599
Ndata_SB_ibin: 115.0 Nbkg_SB_ibin: 127.01320105071004 SFs: 0.9696970732605168


In [19]:
json_str = json.dumps(reweights)

with open(json_name, 'w') as json_file:
    json_file.write(json_str)

In [20]:
with open(json_name,'r', encoding='UTF-8') as f:
     load_dict = json.load(f)

In [21]:
norm_SFs = SF_norm
jsonfile = load_dict
print()
def reweighting_bkg(values):
    
    x_bins = [-1., -0.4, 0.4, 1]
    y_bins = [0., 0.05, 0.1, 1]
    z_bins = [0.4, 0.8, 0.9, 1.]
    w_bins = [0., 0.15, 0.2, 0.4]
    v_bins = [0.01, 0.02, 0.03, 0.08]

    
    
    bins_list = [x_bins, y_bins, z_bins, w_bins, v_bins]

    inbins = 1

    for i in range(len(values)):
        if (values[i] >= max(bins_list[i])) or (values[i] <= min(bins_list[i])):
            inbins = 0

    if inbins:
        bin_indices = np.array([np.digitize(values[i], bins) for i, bins in enumerate(bins_list)])
        bin_number = np.ravel_multi_index(bin_indices - 1, [len(bins) - 1 for bins in bins_list])

        weight = jsonfile['bin_{}'.format(bin_number)][1] * norm_SFs
        #print('nbin:', bin_number, json['bin_{}'.format(bin_number)][0], weight)
    else:
        weight = norm_SFs

    #print(weight)

    return weight

def reweighting_sig(values):
    
    x_bins = [-1., -0.4, 0.4, 1]
    y_bins = [0., 0.05, 0.1, 1]
    z_bins = [0.4, 0.8, 0.9, 1.]
    w_bins = [0., 0.15, 0.2, 0.4]
    v_bins = [0.01, 0.02, 0.03, 0.08]

    
    
    bins_list = [x_bins, y_bins, z_bins, w_bins, v_bins]

    inbins = 1

    for i in range(len(values)):
        if (values[i] >= max(bins_list[i])) or (values[i] <= min(bins_list[i])):
            inbins = 0

    if inbins:
        bin_indices = np.array([np.digitize(values[i], bins) for i, bins in enumerate(bins_list)])
        bin_number = np.ravel_multi_index(bin_indices - 1, [len(bins) - 1 for bins in bins_list])

        weight = jsonfile['bin_{}'.format(bin_number)][1]
        #print('nbin:', bin_number, json['bin_{}'.format(bin_number)][0], weight)
    else:
        weight = 1.0

    #print(weight)

    return weight


In [22]:
bkg_run2 = {}

for bkg in bkg_names+sig_names+['data']:
    bkg_list = []
    for year in years:
        bkg_file = uproot.open("{}/{}/{}.root".format(file_path, bkg, year))
        bkg_tree = bkg_file[tree]
        bkg_arrays = bkg_tree.arrays(library="pd")
        bkg_list.append(pd.DataFrame(bkg_arrays))

    bkg_run2[bkg] = pd.concat(bkg_list, ignore_index=True)

/eos/user/z/zewang/anaconda/anaconda3/envs/hzgml/lib/python3.8/site-packages/uproot/interpretation/library.py:798: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  out[name] = series[name]
/eos/user/z/zewang/anaconda/anaconda3/envs/hzgml/lib/python3.8/site-packages/uproot/interpretation/library.py:798: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  out[name] = series[name]
/eos/user/z/zewang/anaconda/anaconda3/envs/hzgml/lib/python3.8/site-packages/uproot/interpretation/library.py:798: PerformanceWarning: DataFrame is highly fragmen

In [8]:
import pickle
BDT_filename = "/afs/cern.ch/work/z/zewang/private/HZGamma/HiggsZGammaAna/Plotter/Data/UL/BDT/model_HZGamma_BDT_{}_SR.pkl".format(tree)

model = pickle.load(open(BDT_filename, 'rb'))

In [14]:
test = bkg_run2['WZ'][["Z_cos_theta", "lep_cos_theta", "H_relpt", "Z_lead_lepton_eta", "Z_sublead_lepton_eta", "gamma_eta", "lep_phi", "gamma_mvaID", "gamma_ptRelErr", "l1g_deltaR", "l2g_deltaR", "Z_lead_lepton_pt", "Z_sublead_lepton_pt"]].to_numpy()
test_score = model.predict_proba(test)[:, 1]
bkg_run2['WZ']["BDT_score"] = test_score

/tmp/zewang/ipykernel_1392134/1396207503.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  bkg_run2['WZ']["BDT_score"] = test_score


In [23]:
#['Z_cos_theta', 'H_relpt', 'gamma_mvaID', 'gamma_relpt', 'gamma_ptRelErr']
for bkg in bkg_names+sig_names+['data']:
    if bkg == 'data':
        bkg_run2[bkg]["reweight"] = pd.Series([1.0]*len(bkg_run2[bkg]))
    elif bkg in sig_names:
        bkg_run2[bkg]["reweight"] = bkg_run2[bkg][['Z_cos_theta', 'H_relpt', 'gamma_mvaID', 'gamma_relpt', 'gamma_ptRelErr']].apply(reweighting_sig, axis=1)
    else:
        bkg_run2[bkg]["reweight"] = bkg_run2[bkg][['Z_cos_theta', 'H_relpt', 'gamma_mvaID', 'gamma_relpt', 'gamma_ptRelErr']].apply(reweighting_bkg, axis=1)

    #bkg_run2[bkg]["BDT_score"] = model.predict_proba(bkg_run2[bkg][["Z_cos_theta", "lep_cos_theta", "H_relpt", "Z_lead_lepton_eta", "Z_sublead_lepton_eta", "gamma_eta", "lep_phi", "gamma_mvaID", "gamma_ptRelErr", "l1g_deltaR", "l2g_deltaR", "gamma_relpt"]].to_numpy())[:, 1]



/tmp/zewang/ipykernel_3708831/1073733606.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  bkg_run2[bkg]["reweight"] = bkg_run2[bkg][['Z_cos_theta', 'H_relpt', 'gamma_mvaID', 'gamma_relpt', 'gamma_ptRelErr']].apply(reweighting_bkg, axis=1)
/tmp/zewang/ipykernel_3708831/1073733606.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  bkg_run2[bkg]["reweight"] = bkg_run2[bkg][['Z_cos_theta', 'H_relpt', 'gamma_mvaID', 'gamma_relpt', 'gamma_ptRelErr']].apply(reweighting_bkg, axis=1)
/tmp/zewang/ipykernel_3708831/1073733606.py:8: Per

In [14]:
for bkg in bkg_names+sig_names+['data']:
    with uproot.recreate("{}/{}/{}_run2.root".format(file_path, bkg, tree)) as f:
        f[tree] = bkg_run2[bkg]

    #bkg_run2[bkg].to_root("{}/{}/{}_run2.root".format(file_path, bkg, tree), key=tree, mode='a', index=False)



In [27]:
print(sum(bkg_run2['ggH']['weight']*bkg_run2['ggH']['reweight']))
print(sum(bkg_run2['ggH']['weight']))

24.659886087984585
24.665216480715607


In [3]:
file_path = "/eos/user/z/zewang/HZGamma_data/run2UL/skimmed_ntuples"
bkg_names = ['ZG2JToG2L2J', 'WZ',  'TTGJets', 'DYJetsToLL', 'WW', 'TGJets', 'ZGToLLG', 'TT']
sig_names = ['ggH', 'VBF', 'ZH', 'WminusH', 'WplusH', 'ttH']
tree = "zero_to_one_jet"
#tree = "two_jet"
reweight_variables = ['Z_cos_theta', 'H_relpt', 'gamma_mvaID', 'gamma_relpt', 'gamma_ptRelErr']
norm_variables = ['H_mass', 'weight_central']
years = ['2016','2017','2018']
json_name = "/afs/cern.ch/work/z/zewang/private/HZGamma/HiggsZGammaAna/Plotter/Data/reweight_run2UL_{}.json".format(tree)


#bkg_file = uproot.open("/eos/user/z/zewang/HZGamma_data/run2UL/skimmed_ntuples/sig/zero_to_one_jet_run2.root")
#bkg_tree = bkg_file[tree]
#bkg_arrays = bkg_tree.arrays(['H_mass','weight', 'reweight', 'BDT_score'], library="pd")

In [7]:
#data_list = []
#for year in years:
#    data = uproot.open("{}/data/zero_to_one_jet_run2.root".format(file_path))
#    data_tree = data[tree]
#    data_arrays = data_tree.arrays(['H_mass','weight', 'reweight', 'BDT_score'], library="pd")
#    data_list.append(data_arrays)
#    #print(len(data_arrays))

#data_run2 = pd.concat(data_list)
bkg_names = ['ZG2JToG2L2J', 'WZ',  'TTGJets', 'DYJetsToLL', 'WW', 'TGJets', 'ZGToLLG', 'TT']

bkg_list = []
for bkg in bkg_names:
    bkg = uproot.open("{}/{}/zero_to_one_jet_run2.root".format(file_path, bkg))
    bkg_tree = bkg[tree]
    bkg_arrays = bkg_tree.arrays(['H_mass','weight', 'reweight', 'BDT_score'], library="pd")
    bkg_list.append(bkg_arrays)
    #print(len(data_arrays))
bkg_run2_test = pd.concat(bkg_list)

In [8]:
#sum(bkg_arrays['weight']*bkg_arrays['reweight'])
sum(bkg_run2_test['weight'][(bkg_run2_test['BDT_score'] > 0.9150000214576721) & (bkg_run2_test['H_mass'] > 120) & (bkg_run2_test['H_mass'] < 130) ]*bkg_run2_test['reweight'][(bkg_run2_test['BDT_score'] > 0.9150000214576721) & (bkg_run2_test['H_mass'] > 120) & (bkg_run2_test['H_mass'] < 130)])

8.326594603260817

In [6]:
len(bkg_run2_test)

1639291